# Lab 1: KNN for handwritten digits recognition 

**Course:** Introduction to AI (Spring 2025/2026)  
**Instructor:** Yurii Hannich  
**Points:** 6  

---

## 🎯 Goals

In this lab, you will work with the classic **MNIST** dataset (handwritten digits). Unlike Lab 0 (Iris), where we had high-level features (petal length, etc.), here we work with **raw pixels**.

You will:
1.  **Explore the data**: Understand how images are represented as vectors (Flattening).
2.  **Normalize data**: Implement simple scaling and understand why it helps distance-based algorithms.
3.  **Visualise high-dimensional data**: Use unsupervised learning algorithm UMAP for dimensionality reduction to see how 784-dimensional data looks in 2D.
4.  **Train KNN**: Build a classifier for handwritten digits.
5.  **Look inside the box**: Visualize the actual "neighbors" the model uses to make decisions.
6.  **Break the model**: Test how robust the model is to simple pixel shifts.

---

In [ ]:
! pip install numpy matplotlib scikit-learn umap-learn torch torchvision

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import torchvision
from torchvision import datasets, transforms

# Set seed for reproducibility
np.random.seed(42)

## Part 1: Data Loading & Exploration

### 1.1 Loading MNIST with Torchvision
The MNIST dataset consists of 70,000 images of handwritten digits (0-9). Each image is **28x28** pixels.
We will use `torchvision` to download the data locally.

*Note: The first run might take a minute to download.*

In [ ]:
print("Downloading MNIST...")

# Download Training Data
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=None)
# Download Test Data
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=None)

# Extract data as numpy arrays
# X is the image data (N, 28, 28), y is the label (N,)
X_train_raw = train_dataset.data.numpy()
y_train = train_dataset.targets.numpy()

X_test_raw = test_dataset.data.numpy()
y_test = test_dataset.targets.numpy()

print(f"Train X Shape: {X_train_raw.shape}")
print(f"Test X Shape:  {X_test_raw.shape}")
print(f"Train y Shape: {y_train.shape}")
print(f"Test y Shape:  {y_test.shape}")

### 1.2 Visualizing Raw Data
As you can see, we have 2D matrices `(28, 28)` objects in our dataset instead of just vectors (rows). Let's visualize first image to see what we are working with.

In [ ]:
print(X_train_raw[0])

Not the best view.

So lets use matplotlib for it.

In [ ]:
# Plot first 5 images
fig, axes = plt.subplots(1, 5, figsize=(10, 3))
for i in range(5):
    axes[i].imshow(X_train_raw[i], cmap='gray')
    axes[i].set_title(f"Label: {y_train[i]}")
    axes[i].axis('off')
plt.show()

And what about our target?

Let's check it out.

In [ ]:
counts = np.bincount(y_train)

plt.bar(
    np.arange(counts.shape[0]).astype(str), 
    counts
)

plt.xlabel('Class label - Digit')
plt.ylabel('Count of images in train set')
plt.title('Class distribution in train set')

### 🤔 Questions to think about

Before we continue, let's think about what we're dealing with:

1. **What are features in our dataset and what is the dimensionality of the feature space?**  

2. **Is this a classification or regression task?**  

3. **How many classes do we need to predict?**  

## Part 2: Preprocessing

### 2.1 Task 1: Flattening (Forward Reshape) - 1 point
Machine Learning models (like KNN or SVM) typically expect a **feature vector** for each sample, not a 2D matrix. So, we need to convert our images from shape `(N, 28, 28)` to `(N, 784)` as
$28 \times 28 = 784$.

**Task 1:** Implement a function to flatten the image batch.

In [ ]:
def flatten_images(X_2d):
    """
    Converts a batch of images from (N, 28, 28) to (N, 784).
    """
    # N is the number of samples (e.g. 60000)
    N = X_2d.shape[0]

    # YOUR CODE HERE
    # Use .reshape() method for arrays from numpy to change dimensions
    X_flat = ...

    return X_flat

X_train_flat = flatten_images(X_train_raw)
X_test_flat = flatten_images(X_test_raw)

if X_train_flat is not None:
    print(f"Original shape: {X_train_raw.shape}")
    print(f"Flattened shape: {X_train_flat.shape}")

<br/>

**Expected results**:
```
Original shape: (60000, 28, 28)
Flattened shape: (60000, 784)
```

### 2.2 Task 2: Feature Scaling (Normalization) - 1 point

#### 2.2.1 Why do we need scaling?

Machine Learning algorithms, especially those based on **distance** (like KNN, K-Means, or SVMs), are very sensitive to the scale of the data.

Imagine a dataset with two features:

1. **Age:** 20 to 80 years.
2. **Salary:** 10,000 to 200,000 dollars.

When KNN calculates the **Euclidean Distance** between two people:
$$d(p, q) = \sqrt{(Age_1 - Age_2)^2 + (Salary_1 - Salary_2)^2}$$

A small difference in Salary (e.g., $1000) will square to 1,000,000.

A large difference in Age (e.g., 50 years) will square to only 2,500.

**The Result:** The algorithm will completely ignore "Age" because "Salary" numbers are just bigger. The model becomes biased toward features with larger magnitudes.

To fix this, we bring all features to the same scale. We move **from absolute values of feature - to it's variance**, and actually, the **variance is a fuel for ML**.

#### 2.2.2 Common Types of Scaling

There are two main techniques used in Data Science:

##### 1. Min-Max Scaling (Normalization)

This transforms features to fit within a specific range, usually **[0, 1]**.

$$X_{norm} = \frac{X - X_{min}}{X_{max} - X_{min}}$$

##### 2. Standardization (Z-Score Normalization)

This transforms data so that it has a **mean ($\mu$) of 0** and a **standard deviation ($\sigma$) of 1**. This is the most common type of normalization.

$$Z = \frac{X - \mu}{\sigma}$$

#### 2.2.3 What we use for Images

Digital images consists of pixels. In a grayscale image (like MNIST), every pixel is an integer between **0 (black)** and **255 (white)**.

Since the minimum and maximum values are fixed and known ($min=0, max=255$), **Min-Max Scaling** is the standard approach for image data.

The formula simplifies significantly:
$$X_{norm} = \frac{X - 0}{255 - 0} = \frac{X}{255}$$

**Task 2:** Implement this simple normalization in the cell below to convert our pixel values from integers [0, 255] to floats [0.0, 1.0].

In [ ]:
def normalize_data(X):
    """
    Scales input X from [0, 255] to [0, 1].
    """
    # YOUR CODE HERE
    # Ensure the result is float, not integer
    X_norm = ... 

    return X_norm

X_train = normalize_data(X_train_flat)
X_test = normalize_data(X_test_flat)

if X_train is not None:
    print(f"Max value before: {X_train_flat.max()}")
    print(f"Max value after:  {X_train.max()}")

<br/>

**Expected results**:
```
Max value before: 255
Max value after:  1.0
```

## Part 3: Dimensionality Reduction (UMAP)

We now have vectors of size 784. It's impossible for humans to imagine 784-dimensional space.
**UMAP** is a technique to project this high-dimensional data into 2D while preserving clusters (similar digits stay together).

In [ ]:
import umap

# Use a subset of 5000 images for speed
print("Running UMAP... (This may take ~60 seconds)")

if X_train is not None:
    reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42)
    embedding = reducer.fit_transform(X_train[:5000])

    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(embedding[:, 0], embedding[:, 1], c=y_train[:5000], cmap='Spectral', s=5)
    plt.colorbar(scatter, boundaries=np.arange(11)-0.5).set_ticks(np.arange(10))
    plt.title('UMAP projection of MNIST', fontsize=16)
    plt.show()

### 🤔 Questions to think about

Before we continue, let's think about what we're dealing with:

1. **What is dimensionality of the feature space now?**  

2. **Can we run KNN in this, reduced space**  

3. **Look at the close clusters. Do the digits of them really ma look similar?**  

## Part 4: K-Nearest Neighbors

After all the preparation is done - let's classify images using KNN on the **flattened vectors**. Using reduced UMAP-projected is a real option, but in context of this lab we'll make classification in original 784D space.

**Important:** KNN is a "Lazy Learner". It compares the test image to **every** training image (actually there're more optimized versions of KNN that learn some graph structure for faster navigation during inference, but 'vanilla' KNN doesn't do this). 
To save time during this lab, we will use a **subset** of the training data (10,000 samples).


In [ ]:
# Create a smaller subset for speed (10k train, 1k test)
train_limit = 10000
test_limit = 1000

X_train_sub = X_train[:train_limit]
y_train_sub = y_train[:train_limit]

X_test_sub = X_test[:test_limit]
y_test_sub = y_test[:test_limit]

print(f"Training on {len(X_train_sub)} samples")
print(f"Testing on {len(y_test_sub)} samples")

### 4.1 **Task 3:** Implement the training, prediction and evaluation pipeline - 2 points

In [ ]:
def train_and_evaluate(X_train, y_train, X_test, y_test, k=3):
    """
    1. Initialize KNN classifier with n_neighbors=k
    2. Fit on X_train, y_train
    3. Predict X_test
    4. Calculate and return accuracy score
    """
    model = None
    train_accuracy = 0.0
    test_accuracy = 0.0

    # YOUR CODE HERE
    # model = KNeighborsClassifier(...)
    # model.fit(...)
    
    # train_preds = model.predict(...)
    # train_accuracy = accuracy_score(...)
    # test_preds = model.predict(...)
    # test_accuracy = accuracy_score(...)
    
    return model, train_accuracy, test_accuracy

if X_train is not None:
    knn_model, acc_train, acc_test = train_and_evaluate(X_train_sub, y_train_sub, X_test_sub, y_test_sub, k=3)
    print(f"Accuracy with k=3 (train): {acc_train:.4f}")
    print(f"Accuracy with k=3 (test): {acc_test:.4f}")

<br/>

**Expected results**:
```
Accuracy with k=3 (train): 0.9755
Accuracy with k=3 (test): 0.9190
```

### 4.2 **Task 4:** Try different **k** and select the best one - 1 point

**Hyperparameters Tuning**

As KNN is a 'lazy' algorithm - it doesn't have 'parameters' like e.g. neural networks. But it still have hyperparameters.
**Hyperparameters** - are some constants that we set before training started. These are **not parameters of the model, but parameters of training algorithm**.
For KNN we have three of them:
- `n_neighbors` - our **k**
- `weights` - our **voting strategy** - which means that we can assign more weight of vote to closer neighbours.
- `metric` - our **distance metric** - e.g. eucledian (l2), manhatthan (l1) or other.

For the simplicity we will tune just **k** here, check the accuracy (both train and test), evaluate overfitting effect and select best value of **k**.



In [ ]:
def find_best_k(X_train, y_train, X_test, y_test, k_values):
    """
    1. Iterate over k_values.
    2. For each k, train and evaluate the model.
    3. Store train and test accuracies.
    4. Plot the results (Train vs Test accuracy).
    5. Return the best k (based on Test accuracy).
    """
    train_scores = []
    test_scores = []

    # YOUR CODE HERE
        # Loop through k_values
        # Call train_and_evaluate for each k
        # Append scores to lists

    # --- Plotting the results ---
    plt.figure(figsize=(10, 6))
    plt.plot(k_values, train_scores, marker='o', label='Train Accuracy', linestyle='--')
    plt.plot(k_values, test_scores, marker='o', label='Test Accuracy', linewidth=2)
    
    plt.title('Hyperparameter Tuning: Accuracy vs K')
    plt.xlabel('Number of Neighbors (K)')
    plt.ylabel('Accuracy')
    plt.xticks(k_values)
    plt.legend()
    plt.grid(True)
    plt.show()

    # Find the k with the highest test accuracy
    best_index = np.argmax(test_scores)
    best_k = k_values[best_index]
    
    return best_k

# --- Execution ---
k_list = [1, 3, 5, 7, 9, 11, 15, 20]

if 'X_train_sub' in locals():
    best_k = find_best_k(X_train_sub, y_train_sub, X_test_sub, y_test_sub, k_list)
    print(f"The best K based on Test Set is: {best_k}")

<br/>

**Expected results**:
```
The best K based on Test Set is: 1
```

**Analysis**

- As we can see - **overfitting** effect become **smaller** as **k** **increases**.
- But still the highest test score has **k=1**.
- So we can select this value **if the accuracy is the main requirement to our model**.

## Part 5: Inside the Black Box (Visualizing Neighbors)

Accuracy is great, but **why** did the model predict that digit?
We can look at the actual training examples that the model "voted" for.

What is happening here?
1. Finding of the nearest neighbors for a test image.
2. Plotting the query image.
3. Plotting the neighbor images.

In [ ]:
def visualize_neighbors(model, X_test_flat_sample, X_train_flat, y_train, n_neighbors=5):
    """
    X_test_flat_sample: A single flat vector (784,)
    X_train_flat: The training data (N, 784)
    """
    # Reshape input to (1, -1) for sklearn
    query = X_test_flat_sample.reshape(1, -1)

    distances, indices = model.kneighbors(query, n_neighbors=n_neighbors)
    neighbor_indices = indices[0]

    # --- Plotting Code ---
    fig, axes = plt.subplots(1, n_neighbors + 1, figsize=(12, 3))

    # 1. Plot Query Image (Reshape back to 28x28 for visualization)
    axes[0].imshow(X_test_flat_sample.reshape(28, 28), cmap='gray')
    axes[0].set_title("Query")
    axes[0].axis('off')

    # 2. Plot Neighbors
    for i, idx in enumerate(neighbor_indices):
        neighbor_img_flat = X_train_flat[idx]
        neighbor_label = y_train[idx]
     
        # RESHAPE here for plotting
        img_2d = neighbor_img_flat.reshape(28, 28)
        
        axes[i+1].imshow(img_2d, cmap='gray')
        axes[i+1].set_title(f"Neighbor {i+1}\nLabel: {neighbor_label}")
        axes[i+1].axis('off')

    plt.show()

# Visualize neighbors for test image #100
if 'knn_model' in locals() and knn_model is not None:
    test_idx = 6
    print(f"True Label: {y_test_sub[test_idx]}")
    visualize_neighbors(knn_model, X_test_sub[test_idx], X_train_sub, y_train_sub)

As you can see - distinhuish some images may be even hard for humans😅

Let's move now to the last test and the last task.

## Part 6: Breaking the Model (Robustness Test)

We have high accuracy. But does the model "understand" the digit, or just memorize pixel positions?
Let's perform a **Pixel Shift Test**.

### Part 6.1: **Task 5:** Implement a function to shift the image by `n` pixels to the right - 1 point
Since we are working with flat vectors in the model pipeline, your function should:
1. Reshape flat vector to 2D (28, 28).
2. Shift pixels (using `np.roll`).
3. Reshape back to flat vector (784).

In [ ]:
def shift_image_flat(image_flat, shift_amount):
    """
    Shifts the image to the right by shift_amount pixels.
    Input/Output are flat vectors (784,).
    """
    # 1. Reshape to 28x28
    img_2d = image_flat.reshape(28, 28)

    # 2. Shift using np.roll (cyclic shift)
    # YOUR CODE HERE
    shifted_2d = ...

    # 3. Handle wrap-around (fill exposed area with 0)
    if shift_amount > 0:
        shifted_2d[:, :shift_amount] = 0

    # 4. Flatten back to 784
    if shifted_2d is not None:
        return shifted_2d.flatten()
    return image_flat

# --- Experiment ---
if 'knn_model' in locals() and knn_model is not None:
    idx = 100 
    original_vec = X_test_sub[idx]
    true_label = y_test_sub[idx]

    # Shift by 4 pixels
    shift_val = 4
    shifted_vec = shift_image_flat(original_vec, shift_val)

    # Predict
    pred_original = knn_model.predict(original_vec.reshape(1, -1))[0]
    pred_shifted = knn_model.predict(shifted_vec.reshape(1, -1))[0]

    # Visualize
    fig, ax = plt.subplots(1, 2, figsize=(8, 4))
    ax[0].imshow(original_vec.reshape(28, 28), cmap='gray')
    ax[0].set_title(f"Original\nPred: {pred_original}")

    ax[1].imshow(shifted_vec.reshape(28, 28), cmap='gray')
    ax[1].set_title(f"Shifted ({shift_val}px)\nPred: {pred_shifted}")
    plt.show()

In [ ]:
# nearest neighbours for original image
visualize_neighbors(knn_model, original_vec, X_train_sub, y_train_sub)

In [ ]:
# nearest neighbours for shifted by 4 pixels image
visualize_neighbors(knn_model, shifted_vec, X_train_sub, y_train_sub)

In [ ]:
# 1. Combine data: Subset of train + Shifted + Original
# We use a subset of X_train for speed (e.g., 5000 samples)
subset_size = 5000
X_subset = X_train[:subset_size]
y_subset = y_train[:subset_size]

# Stack the shifted and original vectors at the end
X_with_examples = np.vstack([
    X_subset, 
    shifted_vec.reshape(1, -1), 
    original_vec.reshape(1, -1)
])

# 2. Run UMAP
print("Running UMAP on combined data...")
reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42)
embedding = reducer.fit_transform(X_with_examples)

# 3. Separate the coordinates
# The background points (training data)
embedding_background = embedding[:subset_size]

# The last two points are our specific examples
# Index -2 is shifted_vec, Index -1 is original_vec
embedding_shifted = embedding[-2]
embedding_original = embedding[-1]

# 4. Plotting
plt.figure(figsize=(12, 10))

# A) Plot the background training data
scatter = plt.scatter(
    embedding_background[:, 0], 
    embedding_background[:, 1], 
    c=y_subset, 
    cmap='Spectral', 
    s=5, 
    label='Training Data'
)

# B) Plot the Original Point (Start)
plt.scatter(
    embedding_original[0], 
    embedding_original[1], 
    c='black', 
    cmap='Spectral', 
    s=200, 
    marker='*', 
    edgecolors='white', 
    label=f'Original (Label: {true_label})',
    zorder=14 # Draw on top
)

# C) Plot the Shifted Point (End)
plt.scatter(
    embedding_shifted[0], 
    embedding_shifted[1], 
    c='red', 
    s=200, 
    marker='X', 
    edgecolors='white', 
    label='Shifted Image',
    zorder=10
)

# D) Draw the Shift Vector (Arrow)
plt.arrow(
    embedding_original[0], embedding_original[1],      # Start (x, y)
    embedding_shifted[0] - embedding_original[0],      # dx
    embedding_shifted[1] - embedding_original[1],      # dy
    color='black', 
    width=0.05,            # Adjust width based on scale of UMAP
    head_width=0.2,         # Adjust head width
    length_includes_head=True,
    zorder=9
)

plt.title('UMAP Projection: Visualizing the Impact of Pixel Shift', fontsize=16)
plt.colorbar(scatter, boundaries=np.arange(11)-0.5).set_ticks(np.arange(10))
plt.legend(loc='upper right')
plt.grid(True, alpha=0.3)
plt.show()

### 🤔 Questions to think about
**Why did the model fail (or get low confidence) just by moving pixels?**

🗓 You'll learn about it more during next week.